<center><h1>  Exporatory Data Analysis </h1></center>

### Data Info

Historical sales data for 45 Walmart stores located in different regions. Each store contains a number of departments, and the task is predicting the department-wide sales for each store.

In addition, Walmart runs several promotional markdown events throughout the year. These markdowns precede prominent holidays, the four largest of which are the Super Bowl, Labor Day, Thanksgiving, and Christmas. The weeks including these holidays are weighted five times higher in the evaluation than non-holiday weeks.

Part of the challenge is modeling the effects of markdowns on these holiday weeks in the absence of complete/ideal historical data.

- Store - the store number
- Dept - the department number
- Date - the week
- Weekly_Sales -  sales for the given department in the given store
- IsHoliday - whether the week is a special holiday week
- Store - the store number
- Temperature - average temperature in the region
- Fuel_Price - cost of fuel in the region
- MarkDown1-5 - anonymized data related to promotional markdowns that Walmart is running. MarkDown data is only available after Nov 2011, and is not available for all stores all the time. Any missing value is marked with an NA.
- CPI - the consumer price index
- Unemployment - the unemployment rate


For convenience, the four holidays fall within the following weeks in the dataset (not all holidays are in the data):

- **Super Bowl**: 12-Feb-10, 11-Feb-11, 10-Feb-12, 8-Feb-13
- **Labor Day**: 10-Sep-10, 9-Sep-11, 7-Sep-12, 6-Sep-13
- **Thanksgiving**: 26-Nov-10, 25-Nov-11, 23-Nov-12, 29-Nov-13
- **Christmas**: 31-Dec-10, 30-Dec-11, 28-Dec-12, 27-Dec-13

### Getting Data

In [1]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [2]:
train_df = pd.read_csv('sales-forecasting/train.csv')
test_df = pd.read_csv('sales-forecasting/test.csv')
features_df = pd.read_csv('sales-forecasting/features.csv')
stores_df = pd.read_csv('sales-forecasting/stores.csv')

In [3]:
train_df.head()

,Store,Dept,Date,Weekly_Sales,IsHoliday
0,1,1,2010-02-05,24924.50,False
1,1,1,2010-02-12,46039.49,True
2,1,1,2010-02-19,41595.55,False
3,1,1,2010-02-26,19403.54,False
4,1,1,2010-03-05,21827.90,False


In [4]:
features_df.head()

,Store,Date,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,CPI,Unemployment,IsHoliday
0,1,2010-02-05,42.31,2.572,NaN,NaN,NaN,NaN,NaN,211.096358,8.106,False
1,1,2010-02-12,38.51,2.548,NaN,NaN,NaN,NaN,NaN,211.242170,8.106,True
2,1,2010-02-19,39.93,2.514,NaN,NaN,NaN,NaN,NaN,211.289143,8.106,False
3,1,2010-02-26,46.63,2.561,NaN,NaN,NaN,NaN,NaN,211.319643,8.106,False
4,1,2010-03-05,46.50,2.625,NaN,NaN,NaN,NaN,NaN,211.350143,8.106,False


In [5]:
stores_df.head()

,Store,Type,Size
0,1,A,151315
1,2,A,202307
2,3,B,37392
3,4,A,205863
4,5,B,34875


In [6]:
df = pd.merge(train_df, features_df, on=['Store','Date','IsHoliday'], how='inner')
df = pd.merge(df, stores_df, on='Store', how='inner')

### Profile Report

In [7]:
import pandas_profiling

In [8]:
pandas_profiling.ProfileReport(df, minimal=True)

### Visualizations

In [9]:
import ipywidgets as widgets
from ipywidgets import interact, interact_manual
import cufflinks as cf
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot 
init_notebook_mode(connected=True)
cf.go_offline()

In [10]:
dept_store_df = pd.DataFrame(df.groupby(['Date', 'Dept', 'Store'])['Weekly_Sales'].sum()).reset_index()
dept_store_df.Date = pd.to_datetime(dept_store_df['Date'])

In [11]:
def plot_series(plot_type,
                x,
                y,
                up_to_date,
                store,
                dept,
            ):
    date = up_to_date
    title = f'Interactive Dashboard: Weekly Sales'
    if store == 'None' and dept == 'None':
        dept_store_df[(dept_store_df['Date'] <= date)].groupby('Date')['Weekly_Sales'].sum().iplot(kind=plot_type, title=title)
    if store == 'None' and dept != 'None':
        dept_store_df[(dept_store_df['Dept'] == dept) & (dept_store_df['Date'] <= date)].groupby('Date')['Weekly_Sales'].sum().iplot(kind=plot_type, title=title) 
    if store != 'None' and dept == 'None':
        dept_store_df[(dept_store_df['Store'] == store) & (dept_store_df['Date'] <= date)].groupby('Date')['Weekly_Sales'].sum().iplot(kind=plot_type, title=title)
    if store != 'None' and dept != 'None':
        dept_store_df[(dept_store_df['Dept'] == dept) & (dept_store_df['Store'] == store) & (dept_store_df['Date'] <= date)].groupby('Date')['Weekly_Sales'].sum().iplot(kind=plot_type, title=title)    
        
_ = interact(plot_series,
             plot_type = ['line', 'box'],
             store=widgets.Dropdown(options=['None']+list(dept_store_df.Store.unique())),
             dept=widgets.Dropdown(options=['None']+list(dept_store_df.Dept.unique())),
             up_to_date = widgets.DatePicker(value=pd.Timestamp('2012-10-26')),
             x=['Date'],
             y=['Weekly_Sales']
            )

interactive(children=(Dropdown(description='plot_type', options=('line', 'box'), value='line'), Dropdown(descr…

In [12]:
print('Total Weekly Sales Rank per Store')
pd.DataFrame(df.groupby('Store')['Weekly_Sales'].sum().sort_values(ascending=False)).reset_index()

Total Weekly Sales Rank per Store


,Store,Weekly_Sales
0,20,3.013978e+08
1,4,2.995440e+08
2,14,2.889999e+08
3,13,2.865177e+08
4,2,2.753824e+08
5,10,2.716177e+08
6,27,2.538559e+08
7,6,2.237561e+08
8,1,2.224028e+08
9,39,2.074455e+08


In [13]:
print('Total Weekly Sales Rank per Dept')
pd.DataFrame(df.groupby('Dept')['Weekly_Sales'].sum().sort_values(ascending=False)).reset_index()

Total Weekly Sales Rank per Dept


,Dept,Weekly_Sales
0,92,4.839433e+08
1,95,4.493202e+08
2,38,3.931181e+08
3,72,3.057252e+08
4,90,2.910685e+08
...,...,...
76,51,3.057283e+04
77,78,1.714710e+03
78,39,1.779800e+02
79,43,1.432000e+01


In [14]:
print('Max Weekly Sales Info per Store')
df.loc[df.reset_index().groupby(['Store'])['Weekly_Sales'].idxmax()][['Store', 'Dept', 'Date', 'IsHoliday', 'Weekly_Sales']]

Max Weekly Sales Info per Store


,Store,Dept,Date,IsHoliday,Weekly_Sales
6784,1,72,2011-11-25,True,203670.47
13293,2,72,2010-11-26,True,285353.53
20517,3,38,2010-02-05,False,155897.94
36312,4,72,2011-11-25,True,385051.04
45745,5,72,2011-11-25,True,93517.72
51826,6,72,2010-11-26,True,342578.65
65444,7,72,2011-11-25,True,222921.09
71724,8,72,2010-11-26,True,153431.69
84492,9,72,2011-11-25,True,139427.43
90645,10,72,2010-11-26,True,693099.36


In [15]:
print('Max Weekly Sales Info per Dept')
df.loc[df.reset_index().groupby(['Dept'])['Weekly_Sales'].idxmax()][['Dept', 'Store', 'Date', 'IsHoliday', 'Weekly_Sales']]

Max Weekly Sales Info per Dept


,Dept,Store,Date,IsHoliday,Weekly_Sales
120990,1,13,2010-12-24,False,172225.55
94629,2,10,2011-12-23,False,151090.50
123551,3,13,2011-08-26,False,131564.25
36261,4,4,2011-11-25,True,72179.92
90884,5,10,2010-12-24,False,259955.82
...,...,...,...,...,...
128148,95,14,2010-02-05,False,213042.66
274760,96,28,2012-03-09,False,63978.78
17325,97,2,2011-12-23,False,49034.16
131384,98,14,2010-12-24,False,33759.90


In [16]:
holiday_df = pd.DataFrame(train_df[['Date', 'IsHoliday', 'Weekly_Sales']].groupby(['Date', 'IsHoliday'])['Weekly_Sales'].sum()).reset_index()
isholiday_sales_pivot = holiday_df.assign(key=holiday_df.groupby('IsHoliday').cumcount()).pivot('Date','IsHoliday','Weekly_Sales')
isholiday_sales_pivot

IsHoliday,False,True
Date,,
2010-02-05,49750740.50,NaN
2010-02-12,NaN,48336677.63
2010-02-19,48276993.78,NaN
2010-02-26,43968571.13,NaN
2010-03-05,46871470.30,NaN
...,...,...
2012-09-28,43734899.40,NaN
2012-10-05,47566639.31,NaN
2012-10-12,46128514.25,NaN


In [17]:
isholiday_sales_pivot.iplot('box', title=f'Boxplot: IsHoliday x Sales')

Even though there are some outliers when it's not holiday, the difference between medians are not that much.

In [18]:
type_size_pivot = stores_df.assign(key=stores_df.groupby('Type').cumcount()).pivot('Store','Type','Size')
type_size_pivot

Type,A,B,C
Store,,,
1,151315.0,NaN,NaN
2,202307.0,NaN,NaN
3,NaN,37392.0,NaN
4,205863.0,NaN,NaN
5,NaN,34875.0,NaN
6,202505.0,NaN,NaN
7,NaN,70713.0,NaN
8,155078.0,NaN,NaN
9,NaN,125833.0,NaN


In [19]:
type_size_pivot.iplot('box', title=f'Boxplot: Store Type x Size')

In [20]:
group_store_type_sales = df[['Store','Type','Weekly_Sales']].groupby(['Store', 'Type'])['Weekly_Sales'].sum()
group_store_type_sales = pd.DataFrame(group_store_type_sales).reset_index()

type_sales_pivot = group_store_type_sales.assign(key=group_store_type_sales.groupby('Type').cumcount()).pivot('Store','Type','Weekly_Sales')
type_sales_pivot

Type,A,B,C
Store,,,
1,2.224028e+08,NaN,NaN
2,2.753824e+08,NaN,NaN
3,NaN,5.758674e+07,NaN
4,2.995440e+08,NaN,NaN
5,NaN,4.547569e+07,NaN
6,2.237561e+08,NaN,NaN
7,NaN,8.159828e+07,NaN
8,1.299512e+08,NaN,NaN
9,NaN,7.778922e+07,NaN


In [21]:
type_sales_pivot.iplot('box', title=f'Boxplot: Store Type x Sales')

In [22]:
group_type = pd.DataFrame(stores_df.groupby('Type')['Size'].count()).reset_index().rename(columns={'Size':'Count'})
group_type.iplot('pie', labels=['Type'], values='Count', title=f'Type vs Count Piechart')

Almost half of the Stores are from type A. And they are bigger and sell more than the others.

## Pre-processing

### Fixing Date

In [23]:
df['Date'] = pd.to_datetime(df['Date'])
df['Year']=df['Date'].dt.year
df['Month']=df['Date'].dt.month

### Dealing with Null Values

In [24]:
df.isnull().sum()

Store                0
Dept                 0
Date                 0
Weekly_Sales         0
IsHoliday            0
Temperature          0
Fuel_Price           0
MarkDown1       270889
MarkDown2       310322
MarkDown3       284479
MarkDown4       286603
MarkDown5       270138
CPI                  0
Unemployment         0
Type                 0
Size                 0
Year                 0
Month                0
dtype: int64

In [25]:
null_per_store = df[['MarkDown1', 'MarkDown2', 'MarkDown3', 'MarkDown4', 'MarkDown5']].isnull().groupby([df['Store']]).sum().astype(int).reset_index()

In [26]:
null_per_store['ObservedWeeks'] = df.groupby([df['Store']])['Date'].count().tolist()

In [27]:
null_per_store['NullPercentageMarkDown1'] = null_per_store.MarkDown1 * 100 / null_per_store.ObservedWeeks
null_per_store['NullPercentageMarkDown2'] = null_per_store.MarkDown2 * 100 / null_per_store.ObservedWeeks
null_per_store['NullPercentageMarkDown3'] = null_per_store.MarkDown3 * 100 / null_per_store.ObservedWeeks
null_per_store['NullPercentageMarkDown4'] = null_per_store.MarkDown4 * 100 / null_per_store.ObservedWeeks
null_per_store['NullPercentageMarkDown5'] = null_per_store.MarkDown5 * 100 / null_per_store.ObservedWeeks
null_per_store = null_per_store.drop(['MarkDown1', 'MarkDown2', 'MarkDown3', 'MarkDown4', 'MarkDown5'], axis=1)

In [28]:
pd.options.display.float_format = '{:,.2f}'.format
null_per_store

,Store,ObservedWeeks,NullPercentageMarkDown1,NullPercentageMarkDown2,NullPercentageMarkDown3,NullPercentageMarkDown4,NullPercentageMarkDown5
0,1,10244,64.30,70.57,64.97,64.30,64.30
1,2,10238,64.22,70.50,64.92,64.22,64.22
2,3,9036,64.09,72.53,68.95,65.54,64.09
3,4,10272,64.21,69.81,65.61,64.91,64.21
4,5,8999,64.18,74.01,69.77,65.63,64.18
5,6,10211,64.23,69.11,65.64,64.23,64.23
6,7,9762,64.09,73.85,65.52,64.09,64.09
7,8,9895,64.27,70.56,65.65,64.97,64.27
8,9,8867,63.80,76.59,70.20,65.22,63.80
9,10,10315,64.65,75.00,67.45,64.65,64.65


In [29]:
#null_per_date = df[['MarkDown1', 'MarkDown2', 'MarkDown3', 'MarkDown4', 'MarkDown5']].isnull().groupby([df['Date']]).sum().astype(int).reset_index()
def plot_null(x,
              y,
              store,
              markdown,
              ):
    if store == 'None':
        null_per_date = df[markdown].isnull().groupby([df['Date']]).sum().astype(int).reset_index()
    else:
        null_per_date = df[(df['Store'] == store)][markdown].isnull().groupby([df['Date']]).sum().astype(int).reset_index()
    
    null_per_date.iplot('line', x=['Date'], y=[markdown])
        
_ = interact(plot_null,
             store=widgets.Dropdown(options=['None']+list(dept_store_df.Store.unique())),
             x=['Date'],
             y=['NullCounts'],
             markdown=['MarkDown1', 'MarkDown2', 'MarkDown3', 'MarkDown4', 'MarkDown5']
            )

interactive(children=(Dropdown(description='x', options=('Date',), value='Date'), Dropdown(description='y', op…

It's pretty clear that since **2011-11-11** we started to see the discounts (Markdown)

In [30]:
df[df['Date'] < pd.Timestamp('2011-11-11')].isnull().sum()

Store                0
Dept                 0
Date                 0
Weekly_Sales         0
IsHoliday            0
Temperature          0
Fuel_Price           0
MarkDown1       270138
MarkDown2       270138
MarkDown3       270138
MarkDown4       270138
MarkDown5       270138
CPI                  0
Unemployment         0
Type                 0
Size                 0
Year                 0
Month                0
dtype: int64

In [31]:
df[df['Date'] >= pd.Timestamp('2011-11-11')].isnull().sum()

Store               0
Dept                0
Date                0
Weekly_Sales        0
IsHoliday           0
Temperature         0
Fuel_Price          0
MarkDown1         751
MarkDown2       40184
MarkDown3       14341
MarkDown4       16465
MarkDown5           0
CPI                 0
Unemployment        0
Type                0
Size                0
Year                0
Month               0
dtype: int64

In [32]:
df['MarkDown1'] = df['MarkDown1'].fillna(0)
df['MarkDown2'] = df['MarkDown2'].fillna(0)
df['MarkDown3'] = df['MarkDown3'].fillna(0)
df['MarkDown4'] = df['MarkDown4'].fillna(0)
df['MarkDown5'] = df['MarkDown5'].fillna(0)

In [33]:
df.isnull().sum()

Store           0
Dept            0
Date            0
Weekly_Sales    0
IsHoliday       0
Temperature     0
Fuel_Price      0
MarkDown1       0
MarkDown2       0
MarkDown3       0
MarkDown4       0
MarkDown5       0
CPI             0
Unemployment    0
Type            0
Size            0
Year            0
Month           0
dtype: int64

**Note**: I could have used SimpleImputer from sklearn.impute, but as far as I know Markdown is discounts, therefore I assume if they are Null it means 0 discount.

### Final Dataset

In [34]:
type_df = pd.get_dummies(df.Type)
dataset = pd.concat([df, type_df], axis=1)
dataset = dataset.drop(columns='Type')
dataset['IsHoliday'] = dataset.IsHoliday.astype(int)

In [35]:
dataset

,Store,Dept,Date,Weekly_Sales,IsHoliday,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,CPI,Unemployment,Size,Year,Month,A,B,C
0,1,1,2010-02-05,"24,924.50",0,42.31,2.57,0.00,0.00,0.00,0.00,0.00,211.10,8.11,151315,2010,2,1,0,0
1,1,2,2010-02-05,"50,605.27",0,42.31,2.57,0.00,0.00,0.00,0.00,0.00,211.10,8.11,151315,2010,2,1,0,0
2,1,3,2010-02-05,"13,740.12",0,42.31,2.57,0.00,0.00,0.00,0.00,0.00,211.10,8.11,151315,2010,2,1,0,0
3,1,4,2010-02-05,"39,954.04",0,42.31,2.57,0.00,0.00,0.00,0.00,0.00,211.10,8.11,151315,2010,2,1,0,0
4,1,5,2010-02-05,"32,229.38",0,42.31,2.57,0.00,0.00,0.00,0.00,0.00,211.10,8.11,151315,2010,2,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
421565,45,93,2012-10-26,"2,487.80",0,58.85,3.88,"4,018.91",58.08,100.00,211.94,858.33,192.31,8.67,118221,2012,10,0,1,0
421566,45,94,2012-10-26,"5,203.31",0,58.85,3.88,"4,018.91",58.08,100.00,211.94,858.33,192.31,8.67,118221,2012,10,0,1,0
421567,45,95,2012-10-26,"56,017.47",0,58.85,3.88,"4,018.91",58.08,100.00,211.94,858.33,192.31,8.67,118221,2012,10,0,1,0
421568,45,97,2012-10-26,"6,817.48",0,58.85,3.88,"4,018.91",58.08,100.00,211.94,858.33,192.31,8.67,118221,2012,10,0,1,0


In [36]:
# year, month, date, holiday
holidays_info = {
    'SuperBowl': ['2010-02-12', '2011-02-11', '2012-02-10', '2013-02-08'],
    'LaborDay': ['2010-09-10', '2011-09-09', '2012-09-07', '2013-09-06'],
    'Thanksgiving': ['2010-11-26', '2011-11-25', '2012-11-23', '2013-11-29'],
    'Christmas' : ['2010-12-31', '2011-12-30', '2012-12-28', '2013-12-27']
}

In [37]:
holidays_info_df = pd.DataFrame([(j, h[0]) for h in holidays_info.items() for j in h[1]], columns=['Date', 'Holiday']).sort_values(by='Date', ascending=True).reset_index(drop=True)

In [38]:
holidays_info_df['Date'] = pd.to_datetime(holidays_info_df['Date'])
holidays_info_df['Year']=holidays_info_df['Date'].dt.year
holidays_info_df['Month']=holidays_info_df['Date'].dt.month

In [39]:
holidays_info_df

,Date,Holiday,Year,Month
0,2010-02-12,SuperBowl,2010,2
1,2010-09-10,LaborDay,2010,9
2,2010-11-26,Thanksgiving,2010,11
3,2010-12-31,Christmas,2010,12
4,2011-02-11,SuperBowl,2011,2
5,2011-09-09,LaborDay,2011,9
6,2011-11-25,Thanksgiving,2011,11
7,2011-12-30,Christmas,2011,12
8,2012-02-10,SuperBowl,2012,2
9,2012-09-07,LaborDay,2012,9


In [40]:
def get_holiday_countdown(holiday):
    year = min(holidays_info_df.Year)
    serie = (list(holidays_info_df[(holidays_info_df.Holiday == holiday) & (holidays_info_df.Year == year)].Date)[0] - dataset.Date).dt.days
    while serie[serie < 0].any():
        serie[serie < 0] = (list(holidays_info_df[(holidays_info_df.Holiday == holiday) & (holidays_info_df.Year == year)].Date)[0] - dataset.Date).dt.days
        year = year + 1
    print(serie[serie < 0])
    return serie


In [41]:
dataset['ChristmasCountDown'] = get_holiday_countdown('Christmas')
dataset['ThanksgivingCountDown'] = get_holiday_countdown('Thanksgiving')
dataset['LaborDayCountDown'] = get_holiday_countdown('LaborDay')
dataset['SuperBowlCountDown'] = get_holiday_countdown('SuperBowl')

Series([], Name: Date, dtype: int64)
Series([], Name: Date, dtype: int64)
Series([], Name: Date, dtype: int64)
Series([], Name: Date, dtype: int64)


In [42]:
corr = dataset.corr()
corr.style.background_gradient(cmap='coolwarm', axis=None)

,Store,Dept,Weekly_Sales,IsHoliday,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,CPI,Unemployment,Size,Year,Month,A,B,C,ChristmasCountDown,ThanksgivingCountDown,LaborDayCountDown,SuperBowlCountDown
Store,1,0.0240037,-0.085195,-0.000547988,-0.0500969,0.0652902,-0.0598441,-0.0338291,-0.0203307,-0.0427245,-0.0124515,-0.211088,0.208552,-0.182881,0.00299702,0.00101059,-0.0371191,-0.233461,0.439004,-0.00103064,-0.00144318,-0.00155944,0.00041832
Dept,0.0240037,1,0.148032,0.000916102,0.00443652,0.00357198,0.00149414,0.000586856,0.00147547,0.00193731,0.00266803,-0.00747739,0.00783677,-0.00296646,0.00373789,0.000903763,0.0121339,-0.0299676,0.0283237,-0.000881863,0.000208983,-0.00214943,-0.00029886
Weekly_Sales,-0.085195,0.148032,1,0.0127742,-0.00231245,-0.000120296,0.0471723,0.0207165,0.038562,0.0374672,0.0504653,-0.0209213,-0.0258637,0.243828,-0.0101114,0.0284094,0.185417,-0.131219,-0.0953882,-0.0276734,0.0109238,0.00146379,-0.0120038
IsHoliday,-0.000547988,0.000916102,0.0127742,1,-0.155949,-0.0782811,-0.00352061,0.207604,0.266471,0.0115653,-0.0152354,-0.00194351,0.0104595,0.000593038,-0.0567457,0.123376,0.000451024,0.000166554,-0.00101733,-0.128184,0.00299834,0.000823528,-0.320952
Temperature,-0.0500969,0.00443652,-0.00231245,-0.155949,1,0.143859,-0.0264151,-0.179672,-0.0560263,-0.0502809,-0.0147524,0.182112,0.0967302,-0.0583129,0.0658144,0.235983,0.0244925,-0.109032,0.135651,-0.236276,-0.524235,-0.48642,0.221513
Fuel_Price,0.0652902,0.00357198,-0.000120296,-0.0782811,0.143859,1,0.297056,0.0291532,0.0186149,0.166622,0.21542,-0.16421,-0.0338527,0.00336119,0.779633,-0.0408763,-0.0379733,0.0373165,0.00265207,0.0311396,-0.0808692,-0.0951786,0.161818
MarkDown1,-0.0598441,0.00149414,0.0471723,-0.00352061,-0.0264151,0.297056,1,0.174868,-0.0144109,0.838904,0.41505,0.0109148,-0.105168,0.169788,0.501044,-0.0892057,0.0865552,-0.00481437,-0.135776,0.0935719,0.058851,-0.0369964,-0.0178527
MarkDown2,-0.0338291,0.000586856,0.0207165,0.207604,-0.179672,0.0291532,0.174868,1,-0.00608023,0.11325,0.131735,-0.0035539,-0.0414273,0.0783722,0.131867,-0.0193597,0.0408727,-0.00821431,-0.0545108,0.0128122,0.184276,0.102285,-0.149188
MarkDown3,-0.0203307,0.00147547,0.038562,0.266471,-0.0560263,0.0186149,-0.0144109,-0.00608023,1,-0.0120201,0.0424712,-0.00583883,-0.0180776,0.0336414,0.00678897,0.116031,0.0150833,0.00188985,-0.0280724,-0.122323,-0.11615,0.0883175,-0.0912552
MarkDown4,-0.0427245,0.00193731,0.0374672,0.0115653,-0.0502809,0.166622,0.838904,0.11325,-0.0120201,1,0.30337,-0.00204714,-0.0765128,0.127334,0.33534,-0.105569,0.063737,-0.00801238,-0.0927597,0.114168,0.0847859,-0.0331103,-0.0662277


In [44]:
@interact
def correlation(column1=list(dataset.select_dtypes('number').columns),
                column2=list(dataset.select_dtypes('number').columns)
               ):
    print('Correlation: ', dataset[column1].corr(dataset[column2]))

interactive(children=(Dropdown(description='column1', options=('Store', 'Dept', 'Weekly_Sales', 'IsHoliday', '…

In [ ]:
dataset['Weekly_Sales_t_1'] = dataset['Weekly_Sales'].shift(1)
dataset['Weekly_Sales_t_2'] = dataset['Weekly_Sales'].shift(2)
dataset['Weekly_Sales_t_3'] = dataset['Weekly_Sales'].shift(3)

In [45]:
dataset.to_csv('final_dataset.csv', index=False)